## Investigating the Zillow Housing Price Dataset

N. Ranjan & R. Mattson | CSCI 6430* | Mar 11, 2021




TODO: 
- check file names

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2score
from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import mean_absolute_percentage_error as mape
#from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Lasso,Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder

###  Retrieve Data and Prepare Dataset

In [ ]:
#TODO - get from google docs??
DF1B = pd.read_csv(r'Datasets/City_MedianRentalPrice_1Bedroom.csv')
DF2B = pd.read_csv(r'Datasets/City_MedianRentalPrice_2Bedroom.csv')
DF3B = pd.read_csv(r'Datasets/City_MedianRentalPrice_3Bedroom.csv')
DF4B = pd.read_csv(r'Datasets/City_MedianRentalPrice_4Bedroom.csv')

In [ ]:
# remove columns with useless information - These were only indexes
DF1B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF2B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF3B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF4B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)

In [ ]:
# record the number of bedrooms within the frame
DF1B.insert(0, 'BedroomsSq', 1)
DF2B.insert(0, 'BedroomsSq', 2)
DF3B.insert(0, 'BedroomsSq', 3)
DF4B.insert(0, 'BedroomsSq', 4)
#BE CAREFUL WITH INSERTING BEDROOMS
insertBedrooms = np.where(DF1B.columns=='2010-02')[0][0] 
print(int(insertBedrooms))
DF1B.insert(int(insertBedrooms), 'Bedrooms', 1)
DF2B.insert(int(insertBedrooms), 'Bedrooms', 2)
DF3B.insert(int(insertBedrooms), 'Bedrooms', 3)
DF4B.insert(int(insertBedrooms), 'Bedrooms', 4)

In [ ]:
frames = [DF1B,DF2B,DF3B,DF4B]
result = pd.concat(frames)
result = result.sample(frac=1) #Shuffle!
result = result.reset_index(drop=True) #Reset Index

In [ ]:
result

In [ ]:
# result.to_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
result.to_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv')


In [ ]:
del DF1B,DF2B,DF3B,DF4B
del result,frames
#Just saving memory, let's continue with our final dataset

### Preproccess Data

In [ ]:
# DF = pd.read_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
DF = pd.read_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv',index_col=0)
# TrainDF = DF.iloc[:,:-1].copy() # Rest of the columns come here as training data
# TestDF = DF.iloc[:,-1].copy()   # We predicting the last column
# del DF


In [ ]:
DF

In [ ]:
lb=LabelEncoder()
lb.fit(DF['State'])

In [ ]:

RecordAnalysisDF = pd.DataFrame(columns=['RunNumber','Title','Algorithm','R2Mean','R2Median','RMSEMean','RMSEMedian']) #Will save results for later
runNo = 1

In [ ]:
isBedStateOptimized = True

In [ ]:
cut_off = 0.25 # if number of nulls > 30%, just remove the columns
row_count = DF.index.size
NullCountDF = pd.DataFrame({'Null_Count':DF.isnull().sum().to_numpy(),'Col_Name':DF.columns.to_numpy()})
colsToKeep = NullCountDF[NullCountDF.Null_Count<=row_count*cut_off].Col_Name.to_numpy() #columns to keep,rest have too many nulls
DF = DF[colsToKeep].copy() 
print(DF.isnull().sum().to_numpy())
print(colsToKeep)

In [ ]:
#Removing Nulls
def RemoveNulls(TheDF):
   print('Removing Nulls...')
   for col in TheDF.columns:
      for x in TheDF[TheDF[col].isnull()].index: #Going through all columns
         beds = TheDF.loc[x]['Bedrooms']
         state = TheDF.loc[x]['State']
         #print)
         if(isBedStateOptimized == True):
            valFill = TheDF[(TheDF['Bedrooms'] == beds) & (TheDF['State']==state)][col] #Bedroom and state
            if np.all(np.isnan(valFill)):
               valFill = TheDF[(TheDF['Bedrooms'] == beds)][col] #Check if we can do only with bedroom as this is more correlated than state
            if np.all(np.isnan(valFill)):
               valFill = TheDF[(TheDF['State'] == beds)][col] #If only bedrooms didnt work, try with State
            if np.all(np.isnan(valFill)):
               valFill =TheDF[col] #If we get null for those too..just take median of whole column
            valFill = np.nanmedian(valFill)
            TheDF.loc[x,col] = valFill
         else:
            valFill =TheDF[col]
            valFill = np.nanmedian(valFill)
            TheDF.loc[x,col] = valFill
   print('Nulls Removed.')

In [ ]:
RemoveNulls(DF)

In [ ]:
DF

In [ ]:
#Assumed Last Column is Target, it should return Xtrain,Ytrain,Xtest,Ytest
def ChangeDataSetForTimeSeries(TheDF):
    posOfFirstDate = np.where(TheDF.columns=='Bedrooms')[0][0] + 1
    FirstDate = TheDF.columns[posOfFirstDate]
    T = 10
    Startindex = np.where(TheDF.columns==FirstDate)[0][0]
    #print(TheDF.columns[Startindex])
    TrainEndindex = TheDF.columns.size-3
    #ValidationEndindex = TheDF.columns.size-3
    TestEndindex = TheDF.columns.size-2
    #print(TheDF.columns[TrainEndindex])
    size = TrainEndindex - Startindex + 1
    colListTrain = TheDF.columns[0:Startindex].tolist()
    for i in range(T):
        colListTrain.append('T'+str(i))
    TrainDFX = pd.DataFrame(columns=colListTrain[:-1])
    TrainDFY = pd.DataFrame(columns=['T'+str(T)])
    #print(colListTrain,len(colListTrain))
    #Make Train-------------------------------
    X_Arr = []
    Y_Arr = []
    for i in range(len(TheDF.index)): #Row Iteration
        if(i%25==0):
            print('\r', 'Iteration', i+1, ' / Rows:', len(TheDF.index), end='')
        initdataToInsert = TheDF.iloc[i,0:Startindex].to_numpy().tolist()
        #print('-')
        #print(initdataToInsert)
        #print('-')
        for t in range(Startindex,TrainEndindex + 1 - T):
            x = TheDF.iloc[i,t:t+T].to_numpy().tolist()
            #print(x)
            #x = np.concatenate((initdataToInsert,x))
            X_Arr.extend(initdataToInsert)
            X_Arr.extend(x)
            y = TheDF.iloc[i,t+T]
            Y_Arr.append(y)
            #return X_Arr,Y_Arr
        # X_Arr = np.array(X_Arr).reshape(-1,len(colListTrain))
        # Y_Arr = np.array(Y_Arr).reshape(-1,1)
        
        # X_Arr = pd.DataFrame(X_Arr,columns=colListTrain).to_dict()
        # Y_Arr = pd.DataFrame(Y_Arr,columns=['T10'])
        # #print(X_Arr)
        # TrainDFX = TrainDFX.append(X_Arr,ignore_index=True)
        # TrainDFY = TrainDFY.append(Y_Arr,ignore_index=True)
        #break
        #return TrainDFX,TrainDFY
        #for ind in ():
            #print(X)
            #FinalDFX.loc[ind] = X[:,ind]
            #FinalDFY.loc[ind] = Y.reshape(-1,1)
            #break
        #break   
    print('\r', 'Iteration', i+1, ' / Rows:', len(TheDF.index), end='') 
    X_Arr = np.array(X_Arr).reshape(-1,len(colListTrain))
    Y_Arr = np.array(Y_Arr).reshape(-1,1)
    
    X_Arr = pd.DataFrame(X_Arr,columns=colListTrain)
    Y_Arr = pd.DataFrame(Y_Arr,columns=['T10'])
    #print(X_Arr)
    TrainDFX = TrainDFX.append(X_Arr,ignore_index=True)
    TrainDFY = TrainDFY.append(Y_Arr,ignore_index=True)
    #Training Sets Done!!!!!
    testColNames = TheDF.columns[0:Startindex].to_numpy().tolist()
    #valColData = TheDF.columns[0:Startindex].to_numpy().tolist()
    testColData = TheDF.columns[0:Startindex].to_numpy().tolist()
    for i in range(T):
        testColNames.append('T'+str(i))
        #valColData.extend([TheDF.columns[ValidationEndindex-T+i+1]])
        testColData.extend([TheDF.columns[TestEndindex-T+i+1]])
    #Get Data of Validation and Testing.
    print('----')
    #print(valColData,testColData)
    #ValDFX = TheDF[valColData].copy()
    #ValDFX.columns = valAndTestColNames
    #ValDFY = TheDF.iloc[:,ValidationEndindex+1].copy()
    #ValDFY.columns = 'T'+str(T)
    # ValidationEndindex
    TestDFX = TheDF[testColData].copy()
    TestDFX.columns = testColNames
    TestDFY = TheDF.iloc[:,TestEndindex+1].copy()
    TestDFY.columns = 'T'+str(T)
    #return TrainDFX,TrainDFY,ValDFX,ValDFY,TestDFX,TestDFY
    return TrainDFX,TrainDFY,TestDFX,TestDFY
    pass

In [ ]:
TrainDFX,TrainDFY,TestDFX,TestDFY = ChangeDataSetForTimeSeries(DF)

In [ ]:
TrainDFX

In [ ]:
del DF

In [ ]:
#lb = LabelEncoder()

In [ ]:
#Found out Bedroom was more correlated than these and then on 2nd place is State
# TrainDF['RegionName'] = lb.fit_transform(TrainDF['RegionName'])
# TrainDF['State'] = lb.fit_transform(TrainDF['State'])
# TrainDF['Metro'] = lb.fit_transform(TrainDF['Metro'])
# TrainDF['CountyName'] = lb.fit_transform(TrainDF['CountyName'])
#TrainDF['State'] = lb.fit_transform(TrainDF['State'])

In [ ]:
#TrainDF.corr()

In [ ]:
#Pass by Value Reference Magic!
def AddStateSparseMatrix(TrainingDF):
   enc = OneHotEncoder(handle_unknown='ignore',sparse = False)
   SM = enc.fit_transform(TrainingDF['State'].to_numpy().reshape(-1,1))
   for ind,cat in zip(range(0,len(enc.categories_[0])),enc.categories_[0]):
      TrainingDF.insert(0,'is'+str(cat),SM[:,ind])
   TrainingDF.drop(columns=['State'],inplace=True)

### Run Models

In [ ]:
def RunAllModels(TrainDF,Output,title):
    global runNo
    global RecordAnalysisDF
    global isBedStateOptimized
    def PrintMetrics(scores,algo):
        global RecordAnalysisDF
        print('-'*5)
        meanR2 = np.absolute(scores['test_r2']).mean()
        medianR2 = np.median(np.absolute(scores['test_r2']))
        meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
        medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
        meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
        medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))
        print('Average R2 score ', meanR2)
        print('Median R2 score ', medianR2)
        print('Average Root Mean Square Error ', meanRMSE)
        print('Median Root Mean Square Error ', medianRMSE)
        print('-'*5)
        if(isBedStateOptimized==False):
            algo = algo + ' Nulls removed with complete column'
        RecordAnalysisDF = RecordAnalysisDF.append({'RunNumber':runNo,'Title':title,'Algorithm':algo,'R2Mean':meanR2,'R2Median':medianR2,'RMSEMean':meanRMSE,'RMSEMedian':medianRMSE},ignore_index=True)
        #print('Average Mean Absolute Percentage Error ', meanMAPE)
        #print('Median Mean Absolute Percentage Error ', medianMAPE)
    #-----------------------Linear Regression---------------------------------------------
    print(title)
    print('-'*10)
    print('Linear Regression')
    model = make_pipeline(StandardScaler(),LinearRegression())
    # Y_pred = model.predict(X_test)
    # Y_pred = scY.inverse_transform(Y_pred)
    # Y_test = scY.inverse_transform(Y_test)
    scores = cross_validate(model, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    PrintMetrics(scores,'Linear Regression')#,meanMAPE,medianMAPE)
    #----------------------------Polynomial Lasso---------------------------------------
    for _,degree in enumerate([1]):
        polymodel = make_pipeline(StandardScaler(),PolynomialFeatures(degree),Lasso(alpha=1e-2,max_iter=1e+6,tol=1e-2))
        scores = cross_validate(polymodel, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
        print('Polynomial Degree with Lasso: '+str(degree))
        PrintMetrics(scores,'Lasso Degree ' + str(degree))

        polymodel = make_pipeline(StandardScaler(),PolynomialFeatures(degree),Ridge(alpha=1e-3,max_iter=1e+6))
        scores = cross_validate(polymodel, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
        print('Polynomial Degree with Ridge: '+str(degree))
        PrintMetrics(scores,'Ridge Degree ' + str(degree))
    #----------------------------DecisionTreeRegressor---------------------------------------
    model = make_pipeline(StandardScaler(),DecisionTreeRegressor(max_depth=100))
    scores = cross_validate(model, TrainDF, Output, cv=15,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    print('DecisionTreeRegressor')
    PrintMetrics(scores,'DecisionTreeRegressor')
    runNo = runNo + 1
    print('-'*5)
    print('-'*15)

### Method 1: Drop all the Categorical columns

In [ ]:
FinalTrainDF = TrainDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
# RemoveNulls(FinalTrainDF,True)
FinalTrainDF.drop(columns=['State'],inplace=True)
RunAllModels(FinalTrainDF,TrainDFY,'Dropped Categorical Data')

### Method2: Next, Let's change States to a sparse matrix and then to 51 columns, 1 for each State. Lets drop the other categorical columns

In [ ]:
FinalTrainDF = TrainDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
#RemoveNulls(FinalTrainDF,True)
AddStateSparseMatrix(FinalTrainDF)
#FinalTrainDF.drop(columns=['State'],inplace=True)
RunAllModels(FinalTrainDF,TrainDFY,'One Hot Encoded State,Removed Others')
#FinalTrainDF

In [ ]:
RecordAnalysisDF

In [ ]:
RecordAnalysisDF.index +=1

In [ ]:
RecordAnalysisDF.to_csv('RecordAnalysis.csv',index_label='S.No.')

In [ ]:
#After Analyzing this CSV File (Copy of the table is in the report), we found that "One Hot Encoded State,Removed Others - NAN removed with column with Linear Regression" had the least R2Mean and RMSE Median. So, it had performed best in the 2/4 metrics we considered. Hence, we decided to go ahead with it.

### Testing

In [ ]:
#One Hot Encoded State,Removed Others	Linear Regression
FinalTrainDF = TrainDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
#RemoveNulls(FinalTrainDF,True)
AddStateSparseMatrix(FinalTrainDF)
FinalTestDF = TestDFX.drop(columns=['RegionName', 'Metro', 'CountyName'])
#FinalTrainDF[ValDF.name] = ValDF
#RemoveNulls(FinalTrainDF,False)
AddStateSparseMatrix(FinalTestDF)
#FinalTrainDF.drop(columns=['State'],inplace=True)

In [ ]:
model = LinearRegression()
hist = model.fit(FinalTrainDF,TrainDFY)
YPred = model.predict(FinalTestDF)

In [ ]:
R2Score = r2score(TestDFY,YPred)
RMSE = mse(TestDFY,YPred,squared=False)
print('R2 score ', R2Score)
print('Root Mean Square Error ', RMSE)
print('Average of Output ',TestDFY.mean())

In [ ]:
lb.classes_

In [ ]:
for bed in range(1,5):
    filterArr = (FinalTestDF['Bedrooms'] == bed)
    #print(filterArr)
    print('For '+ str(bed) +' bedrooms')
    print('-'*5)
    R2Score = r2score(TestDFY[filterArr],YPred[filterArr])
    RMSE = mse(TestDFY[filterArr],YPred[filterArr],squared=False)
    print('R2 score ', R2Score)
    print('Root Mean Square Error ', RMSE)
    print('Average of '+ str(bed) +' Bedroom Cost ',TestDFY[filterArr].mean())
    print('-'*10)